# PyTorch BigBird NER Baseline - CV 0.615
This notebook is a PyTorch starter notebook for Kaggle's "Feedback Prize - Evaluating Student Writing" Competition. It demonstrates how to train, infer, and submit a model to Kaggle without internet. Currently this notebook uses

* backbone BigBird  (with HuggingFace's head for TokenClassification)
* NER formulation (with `is_split_into_words=True` tokenization)
* one fold

By changing a few lines of code, we can use this notebook to evaluate different PyTorch backbones! And we can run all sorts of other experiments. If we try a backbone that doesn't accept 1024 wide tokens (like BigBird or LongFormer), then we can add a sliding window to train and inference. BigBird is a new SOTA transformer with arXiv paper [here][3] which can accept large token inputs as wide as 4096!

The model in this notebook uses HuggingFace's `AutoModelForTokenClassification`. If we want a custom head, we could use `AutoModel` and then build our own head. See my TensorFlow notebook [here][2] for an example.

The tokenization process uses `tokenizer(txt.split(), is_split_into_words=True)`, note that this ignores characters like `\n`. If we want our model to see new paragraphs, we need to rewrite this code and avoid `is_split_into_words=True`. See my TensorFlow notebook [here][2] for an example.

This notebook uses many code cells from Raghavendrakotala's great notebook [here][1]. Don't forget to upvote Raghavendrakotala's notebook :-)

[1]: https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
[2]: https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-617
[3]: https://arxiv.org/abs/2007.14062

# Configuration
This notebook can either train a new model or load a previously trained model (made from previous notebook version). Furthermore, this notebook can either create new NER labels or load existing NER labels (made from previous notebook version). In this notebook version, we will load model and load NER labels.

Also this notebook can load huggingface stuff (like tokenizers) from a Kaggle dataset, or download it from internet. (If it downloads from internet, you can then put it in a Kaggle dataset, so next time you can turn internet off).

In [69]:
from tqdm import tqdm

In [70]:
import os
# DECLARE HOW MANY GPUS YOU WISH TO USE. 
# KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# VERSION FOR SAVING MODEL WEIGHTS
TRAIN_VER = 1
LOAD_VER = 26

# IF VARIABLE IS NONE, THEN NOTEBOOK COMPUTES TOKENS
# OTHERWISE NOTEBOOK LOADS TOKENS FROM PATH
LOAD_TOKENS_FROM = 'NER_token'

# IF VARIABLE IS NONE, THEN NOTEBOOK TRAINS A NEW MODEL
# OTHERWISE IT LOADS YOUR PREVIOUSLY TRAINED MODEL
# LOAD_MODEL_FROM = 'trained_model'
LOAD_MODEL_FROM = None

# IF FOLLOWING IS NONE, THEN NOTEBOOK 
# USES INTERNET AND DOWNLOADS HUGGINGFACE 
# CONFIG, TOKENIZER, AND MODEL
# DOWNLOADED_MODEL_PATH = '../input/py-bigbird-v26' 
DOWNLOADED_MODEL_PATH = None

if DOWNLOADED_MODEL_PATH is None:
    DOWNLOADED_MODEL_PATH = 'model'    
MODEL_NAME = 'google/bigbird-roberta-base'

In [71]:
from torch import cuda
import torch

config = {'model_name': MODEL_NAME,
         'max_length': 1024,
         # 'train_batch_size':4,
         # 'valid_batch_size':4,
         'train_batch_size':1,
         'valid_batch_size':3,
         'epochs':5,
         'learning_rates': [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7],
         'max_grad_norm':10,
         'device':  'cuda' if cuda.is_available() else 'cpu'
         # 'device':  'cpu'
          }

# THIS WILL COMPUTE VAL SCORE DURING COMMIT BUT NOT DURING SUBMIT
COMPUTE_VAL_SCORE = True
if len( os.listdir('../data/test') ) > 5:
      COMPUTE_VAL_SCORE = False

# How To Submit PyTorch Without Internet
Many people ask me, how do I submit PyTorch models without internet? With HuggingFace Transformer, it's easy. Just download the following 3 things (1) model weights, (2) tokenizer files, (3) config file, and upload them to a Kaggle dataset. Below shows code how to get the files from HuggingFace for Google's BigBird-base. But this same code can download any transformer, like for example roberta-base.

In [72]:
from transformers import *
if DOWNLOADED_MODEL_PATH == 'model' and not os.path.exists('model'):
    os.mkdir('model')

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
    tokenizer.save_pretrained('model')

    config_model = AutoConfig.from_pretrained(MODEL_NAME)
    config_model.num_labels = 15
    config_model.save_pretrained('model')

    backbone = AutoModelForTokenClassification.from_pretrained(MODEL_NAME,
                                                               config=config_model)
    backbone.save_pretrained('model')

# Load Data and Libraries
In addition to loading the train dataframe, we will load all the train and text files and save them in a dataframe.

In [73]:
import numpy as np, os 
import pandas as pd, gc 
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import accuracy_score

In [74]:
train_df = pd.read_csv('../data/train.csv')
print( train_df.shape )
train_df.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [75]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, test_texts = [], []
for f in list(os.listdir('../data/test')):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../data/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts.head()

,id,text
0,0FB0700DAF44,"During a group project, have you ever asked a ..."
1,18409261F5C2,80% of Americans believe seeking multiple opin...
2,D46BCB48440A,"When people ask for advice,they sometimes talk..."
3,D72CB1C11673,Making choices in life can be very difficult. ...
4,DF920E0A7337,Have you ever asked more than one person for h...


In [76]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../data/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../data/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

100%|██████████| 15594/15594 [00:01<00:00, 12302.38it/s]


,id,text
0,0000D23A521A,"Some people belive that the so called ""face"" o..."
1,00066EA9880D,Driverless cars are exaclty what you would exp...
2,000E6DE9E817,Dear: Principal\n\nI am arguing against the po...
3,001552828BD0,Would you be able to give your car up? Having ...
4,0016926B079C,I think that students would benefit from learn...


# Convert Train Text to NER Labels
We will now convert all text words into NER labels and save in a dataframe.

In [77]:
if not os.path.exists(f'{LOAD_TOKENS_FROM}/train_NER.csv'):
    all_entities = []
    for ii, i in enumerate(train_text_df.iterrows()):
        if ii%100==0: print(ii, ', ' ,end='')
        total = i[1]['text'].split().__len__()
        entities = ["O"] * total
        # print(total)
        for j in train_df[train_df['id'] == i[1]['id']].iterrows():
            discourse = j[1]['discourse_type']
            list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
            entities[list_ix[0]] = f"B-{discourse}"
            for k in list_ix[1:]: 
                # entities[k] = f"I-{discourse}"
                try:
                    entities[k] = f"I-{discourse}"
                except:
                    print(len(entities))
                    print(k)
        all_entities.append(entities)
    train_text_df['entities'] = all_entities
    train_text_df.to_csv('train_NER.csv',index=False)
    
else:
    from ast import literal_eval
    train_text_df = pd.read_csv(f'{LOAD_TOKENS_FROM}/train_NER.csv')
    # pandas saves lists as string, we must convert back
    train_text_df.entities = train_text_df.entities.apply(lambda x: literal_eval(x) )
    
print( train_text_df.shape )
train_text_df.head()

(15594, 3)


,id,text,entities
0,E112CE5938F4,Segoing Cowboys is a good program that I think...,"[B-Position, I-Position, I-Position, I-Positio..."
1,13326EED1708,"Yes I agree that Venus is dangerous place, but...","[B-Position, I-Position, I-Position, I-Positio..."
2,DD3F7252F887,Facial action coding sytsem should be used bec...,"[B-Position, I-Position, I-Position, I-Positio..."
3,E676D0FB128A,"My fellow citizens, there are many reasons why...","[O, O, O, B-Position, I-Position, I-Position, ..."
4,9A03E46C0A51,"Dear Principal,\n\nI believe that changing the...","[O, O, B-Position, I-Position, I-Position, I-P..."


In [78]:
# CREATE DICTIONARIES THAT WE CAN USE DURING TRAIN AND INFER
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [79]:
labels_to_ids

{'O': 0,
 'B-Lead': 1,
 'I-Lead': 2,
 'B-Position': 3,
 'I-Position': 4,
 'B-Claim': 5,
 'I-Claim': 6,
 'B-Counterclaim': 7,
 'I-Counterclaim': 8,
 'B-Rebuttal': 9,
 'I-Rebuttal': 10,
 'B-Evidence': 11,
 'I-Evidence': 12,
 'B-Concluding Statement': 13,
 'I-Concluding Statement': 14}

In [80]:
# text = train_dataset['text'][0]

In [81]:
# pre_text = '[CLS] ' + text.replace('.', '. [SEP]')
# encoding = tokenizer(pre_text.split(),
#                     is_split_into_words=True,
#                     add_special_tokens=False,
#                     return_offsets_mapping=True, 
#                     padding='max_length', 
#                     truncation=True, 
#                     max_length=1024)

In [82]:
# encoding.word_ids()

In [83]:
# encoding['input_ids']

# Define the dataset function
Below is our PyTorch dataset function. It always outputs tokens and attention. During training it also provides labels. And during inference it also provides word ids to help convert token predictions into word predictions.

Note that we use `text.split()` and `is_split_into_words=True` when we convert train text to labeled train tokens. This is how the HugglingFace tutorial does it. However, this removes characters like `\n` new paragraph. If you want your model to see new paragraphs, then we need to map words to tokens ourselves using `return_offsets_mapping=True`. See my TensorFlow notebook [here][1] for an example.

Some of the following code comes from the example at HuggingFace [here][2]. However I think the code at that link is wrong. The HuggingFace original code is [here][3]. With the flag `LABEL_ALL` we can either label just the first subword token (when one word has more than one subword token). Or we can label all the subword tokens (with the word's label). In this notebook version, we label all the tokens. There is a Kaggle discussion [here][4]

[1]: https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-617
[2]: https://huggingface.co/docs/transformers/custom_datasets#tok_ner
[3]: https://github.com/huggingface/transformers/blob/86b40073e9aee6959c8c85fcba89e47b432c4f4d/examples/pytorch/token-classification/run_ner.py#L371
[4]: https://www.kaggle.com/c/feedback-prize-2021/discussion/296713

In [84]:
import  re
LABEL_ALL_SUBTOKENS = True

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, get_wids):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.get_wids = get_wids # for validation

    def __getitem__(self, index):
        # GET TEXT AND WORD LABELS
        text = self.data.text[index]
        word_labels = self.data.entities[index] if not self.get_wids else None
        text = text.strip()
        text = re.sub(r'\s{2,}', '. ', text)
        text = re.sub(r'\xa0', ' ', text)
        text = re.sub(r'\.{6}','',text)
        text = re.sub(r'\.\.', '.', text)
        text = re.sub(r'\. ', '. [SEP] [CLS] ' ,text)
        text = re.sub(r'! ', '! [SEP] [CLS] ' ,text)
        text = re.sub(r'\? ', '? [SEP] [CLS] ' ,text)
        # text = text[:-7]
        # text= "[CLS] " + text

        # TOKENIZE TEXT
        encoding = self.tokenizer(text.split(),
                             is_split_into_words=True,
                             add_special_tokens= True,
                             #return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        word_ids = encoding.word_ids()  
        
        # CREATE TARGETS
        if not self.get_wids:
            previous_word_idx = None
            previous_cls_ids = 0
            sen_count = 0
            label_ids = []
            sen_label_ids = []
            input_ids = encoding['input_ids']
            for i, word_idx in enumerate(word_ids):                            
                if word_idx is None:
                    label_ids.append(-100)
                elif input_ids[i] == 66: # SEP
                    label_ids.append(-100)
                    sen_count += 1
                    if len(sen_label_ids) != 0:
                        label_ids[previous_cls_ids] = max(sen_label_ids, key=sen_label_ids.count)
                    else:
                        label_ids[previous_cls_ids] = -100
                elif input_ids[i] == 65: # CLS
                    previous_cls_ids = i
                    label_ids.append(-1) # wait for filling
                    sen_label_ids = []
                    sen_count += 1
                elif word_idx != previous_word_idx:              
                    label_ids.append( labels_to_ids[word_labels[word_idx-sen_count]] )
                    sen_label_ids.append(label_ids[-1])
                else:
                    if LABEL_ALL_SUBTOKENS:
                        label_ids.append( labels_to_ids[word_labels[word_idx-sen_count]] )
                        sen_label_ids.append(label_ids[-1])
                    else:
                        label_ids.append(-100)
                previous_word_idx = word_idx
            if len(sen_label_ids) != 0:
                label_ids[previous_cls_ids]= max(sen_label_ids, key=sen_label_ids.count)
            else:
                label_ids[previous_cls_ids] = -100
            encoding['labels'] = label_ids
        # CONVERT TO TORCH TENSORS
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        if self.get_wids: 
            word_ids2 = [w if w is not None else -1 for w in word_ids]
            item['wids'] = torch.as_tensor(word_ids2)
        
        return item

    def __len__(self):
        return self.len

# Create Train and Validation Dataloaders
We will use the same train and validation subsets as my TensorFlow notebook [here][1]. Then we can compare results. And/or experiment with ensembling the validation fold predictions.

[1]: https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-617

In [85]:
# CHOOSE VALIDATION INDEXES (that match my TF notebook)
IDS = train_df.id.unique()
print('There are',len(IDS),'train texts. We will split 90% 10% for validation.')

# TRAIN VALID SPLIT 90% 10%
np.random.seed(42)
train_idx = np.random.choice(np.arange(len(IDS)),int(0.9*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)
np.random.seed(None)

There are 15594 train texts. We will split 90% 10% for validation.


In [86]:
import re
text = train_text_df.loc[train_idx[13],'text']
text = text.strip()
text = re.sub(r'\.\s{2,}', '. ', text)
text = re.sub(r'\.{6}','',text)
text = re.sub(r'\. ', '. [SEP] [CLS] ' ,text)
text = re.sub(r'! ', '! [SEP] [CLS] ' ,text)
text = re.sub(r'\? ', '? [SEP] [CLS] ' ,text)
# text = text[:-7]
text= "[CLS] " + text +" [SEP]"
text

"[CLS] Phones and Driving, should it be a thing? [SEP] [CLS] Today over 5 billion people own a mobile device, and a little over 1 billion own some sort of vehicle. [SEP] [CLS] These are two of the things people feel they need in life. [SEP] [CLS] But maybe the use of both at once is a bit too overwhelming. [SEP] [CLS] Driving while texting, or even making a phone call is excessive. [SEP] [CLS] It can be dangerous and hurt people in multiple ways. [SEP] [CLS] Phone usage while driving should be prohibited in every state, country and continent!\n\nUsing a phone while driving a vehicle should not be allowed anywhere! [SEP] [CLS] It can cause many accidents. [SEP] [CLS] It can also bring troubles and pain to ones family. [SEP] [CLS] For example, in February 2018, a 17 year old in Alabaster, Alabama killed a ten year old and her father in an accident because the teen was texting while driving. [SEP] [CLS] If the teen would have been paying attention to the road and not their phone the accid

In [87]:
# CREATE TRAIN SUBSET AND VALID SUBSET
data = train_text_df[['id','text', 'entities']]
train_dataset = data.loc[data['id'].isin(IDS[train_idx]),['text', 'entities']].reset_index(drop=True)
test_dataset = data.loc[data['id'].isin(IDS[valid_idx])].reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

tokenizer = AutoTokenizer.from_pretrained(DOWNLOADED_MODEL_PATH) 
training_set = dataset(train_dataset, tokenizer, config['max_length'], False)
testing_set = dataset(test_dataset, tokenizer, config['max_length'], True)

FULL Dataset: (15594, 3)
TRAIN Dataset: (14034, 2)
TEST Dataset: (1560, 3)


In [88]:
# TRAIN DATASET AND VALID DATASET
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                # 'num_workers': 2,
                'num_workers': 0,
                'pin_memory':True
               }

test_params = {'batch_size': config['valid_batch_size'],
               'shuffle': False,
               # 'num_workers': 2,
               'num_workers': 0,
               'pin_memory':True
              }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# TEST DATASET
test_texts_set = dataset(test_texts, tokenizer, config['max_length'], True)
test_texts_loader = DataLoader(test_texts_set, **test_params)

In [89]:
text = train_dataset.loc[692,'text']
text = text.strip()
text = re.sub(r'\s{2,}', '. ', text)
text = re.sub(r'\xa0', ' ', text)
text = re.sub(r'\.{6}','',text)
text = re.sub(r'\.\.', '.', text)
text = re.sub(r'\. ', '. [SEP] [CLS] ' ,text)
text = re.sub(r'! ', '! [SEP] [CLS] ' ,text)
text = re.sub(r'\? ', '? [SEP] [CLS] ' ,text)
# text = text[:-7]
text= "[CLS] " + text +" [SEP]"
text

"[CLS] Every day millions of children come home on a bus instead of staying after school, and for many, there is a very good reason. [SEP] [CLS] It seems to make sense, having everyone take a extra curricular activity, but in reality it doesn't work. [SEP] [CLS] Every student has a reason to go, or not to go to an extra curricular activity and some reasons are not just because of preference. [SEP] [CLS] Not everyone has a lifestyle that allows them to stay after school or do a sport, many of the students won't have a way to get home. [SEP] [CLS] The school will not be able to provide enough buses for these students either. [SEP] [CLS] There isn't even enough room in the current activity rooms to hold 78 kids, and the materials needed are to expensive. [SEP] [CLS] Having mandatory extra curricular activities will be a disaster because there is not enough room, it is to expensive, and students won't be able to get home. [SEP] [CLS] If our school system made extra curricular activities ma

In [90]:
tokenizer.decode(training_set[20]['input_ids'])

"[CLS] Fellow Citizens, there are many reasons why limiting car usgage has outstanding oppritunities and advantages.[SEP][CLS] For an example, If there aren't people out there spending their hard earned money that they worked for, for an expensive car that they are just going to use to get to someplace faster when they could just stop being lazy for once and walk or run, ride bikes, hike, or take a bus, instead of spending money after money to fill a car up to just waste gas, and to fill it up again.[SEP][CLS] Running, riding bikes, hiking, etc.[SEP][CLS] gives you a chance to experience reality.[SEP][CLS] Being outside gives people a chance to see the earths real beauty, and to smell the fresh air.[SEP][CLS] Limiting car usage helps people stay in shape and keeps them healthy if their not just sitting in a car for half a hour when they could be running or walking.[SEP][CLS] Residents in Vauban, Germany are permitted car ownership, but there are only two places to park : large garages 

In [91]:
train_dataset.loc[692,'text']

"Every day millions of children come home on a bus instead of staying after school, and for many, there is a very good reason. It seems to make sense, having everyone take a extra curricular activity, but in reality it doesn't work. Every student has a reason to go, or not to go to an extra curricular activity and some reasons are not just because of preference. Not everyone has a lifestyle that allows them to stay after school or do a sport, many of the students won't have a way to get home. The school will not be able to provide enough buses for these students either. There isn't even enough room in the current activity rooms to hold 78 kids, and the materials needed are to expensive. Having mandatory extra curricular\xa0activities will be a disaster because there is not enough room, it is to expensive, and students won't be able to get home.\n\nIf our school system made extra curricular activities mandatory the first problem would be the amount of space needed to support the student

In [92]:
# for i in range(len(train_idx)):
#     try:
#         res = sum(training_set[i]['labels']==-1)
#         if res!=0:
#             print('-1 label')
#             print(i)
#     except Exception as e:
#         print(e)
#         print(i)

# training_set[14]

In [93]:
# for i in range(14034):
#     try:
#         x = training_set[i]
#     except:
#         print(i)

In [94]:
tokenizer.decode(training_set[692]['input_ids'])

"[CLS] Every day millions of children come home on a bus instead of staying after school, and for many, there is a very good reason.[SEP][CLS] It seems to make sense, having everyone take a extra curricular activity, but in reality it doesn't work.[SEP][CLS] Every student has a reason to go, or not to go to an extra curricular activity and some reasons are not just because of preference.[SEP][CLS] Not everyone has a lifestyle that allows them to stay after school or do a sport, many of the students won't have a way to get home.[SEP][CLS] The school will not be able to provide enough buses for these students either.[SEP][CLS] There isn't even enough room in the current activity rooms to hold 78 kids, and the materials needed are to expensive.[SEP][CLS] Having mandatory extra curricular activities will be a disaster because there is not enough room, it is to expensive, and students won't be able to get home.[SEP][CLS] If our school system made extra curricular activities mandatory the fi

In [95]:
training_set[0]['labels'][18]

tensor(4)

In [96]:
training_set[0]['labels'][:500]

tensor([   4,    3,    3,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4, -100,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4, -100,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4, -100,    6,    5,    6,    6,    6,    6,    6,
           6,    6,    6,    6,    6, -100,   12,   11,   12,   12,   12,   12,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12, -100,    6,
           0,    0,    0,    0,    5,    6,    6,    6,    6,    6,    6,    6,
           6,    6,    6, -100,   12,   11,   12,   12,   12,   12,   12, -100,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
        -100,    0,    0,    0,    0,   

# Train Model
The PyTorch train function is taken from Raghavendrakotala's great notebook [here][1]. I assume it uses a masked loss which avoids computing loss when target is `-100`. If not, we need to update this.

In Kaggle notebooks, we will train our model for 5 epochs `batch_size=4` with Adam optimizer and learning rates `LR = [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7]`. The loaded model was trained offline with `batch_size=8` and `LR = [5e-5, 5e-5, 5e-6, 5e-6, 5e-7]`. (Note the learning rate changes `e-5`, `e-6`, and `e-7`). Using `batch_size=4` will probably achieve a better validation score than `batch_size=8`, but I haven't tried yet.

[1]: https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533

In [97]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    #tr_preds, tr_labels = [], []
    
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader, total=len(training_loader))):
        
        ids = batch['input_ids'].to(config['device'], dtype = torch.long)
        mask = batch['attention_mask'].to(config['device'], dtype = torch.long)
        labels = batch['labels'].to(config['device'], dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                               return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 1000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss after {idx:04d} training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        #tr_labels.extend(labels)
        #tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=config['max_grad_norm']
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [98]:
# CREATE MODEL
config_model = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json') 
model = AutoModelForTokenClassification.from_pretrained(
                   DOWNLOADED_MODEL_PATH+'/pytorch_model.bin',config=config_model)
model.to(config['device'])
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rates'][0])

In [99]:
## ignore __floordiv__ is deprecated
import warnings
warnings.filterwarnings("ignore")

# LOOP TO TRAIN MODEL (or load model)
if not LOAD_MODEL_FROM:
    for epoch in range(config['epochs']):
        
        print(f"### Training epoch: {epoch + 1}")
        for g in optimizer.param_groups: 
            g['lr'] = config['learning_rates'][epoch]
        lr = optimizer.param_groups[0]['lr']
        print(f'### LR = {lr}\n')
        
        train(epoch)
        torch.cuda.empty_cache()
        gc.collect()
        
    torch.save(model.state_dict(), f'bigbird_v{TRAIN_VER}.pt')
else:
    model.load_state_dict(torch.load(f'{LOAD_MODEL_FROM}/bigbird_v{LOAD_VER}.pt'))
    print('Model loaded.')

### Training epoch: 1
### LR = 2.5e-05



  0%|          | 0/14034 [00:00<?, ?it/s]

Training loss after 0000 training steps: 2.6564600467681885


  1%|▏         | 183/14034 [01:13<1:32:30,  2.50it/s]


KeyboardInterrupt: 

# Inference and Validation Code
We will infer in batches using our data loader which is faster than inferring one text at a time with a for-loop. The metric code is taken from Rob Mulla's great notebook [here][2]. Our model achieves validation F1 score 0.615! 

During inference our model will make predictions for each subword token. Some single words consist of multiple subword tokens. In the code below, we use a word's first subword token prediction as the label for the entire word. We can try other approaches, like averaging all subword predictions or taking `B` labels before `I` labels etc.

[1]: https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
[2]: https://www.kaggle.com/robikscube/student-writing-competition-twitch

In [ ]:
def inference(batch):
                
    # MOVE BATCH TO GPU AND INFER
    ids = batch["input_ids"].to(config['device'])
    mask = batch["attention_mask"].to(config['device'])
    outputs = model(ids, attention_mask=mask, return_dict=False)
    all_preds = torch.argmax(outputs[0], axis=-1).cpu().numpy() 

    # INTERATE THROUGH EACH TEXT AND GET PRED
    predictions = []
    for k,text_preds in enumerate(all_preds):
        token_preds = [ids_to_labels[i] for i in text_preds]

        prediction = []
        word_ids = batch['wids'][k].numpy()  
        previous_word_idx = -1
        for idx,word_idx in enumerate(word_ids):                            
            if word_idx == -1:
                pass
            elif word_idx != previous_word_idx:              
                prediction.append(token_preds[idx])
                previous_word_idx = word_idx
        predictions.append(prediction)
    
    return predictions

In [ ]:
# https://www.kaggle.com/zzy990106/pytorch-ner-infer
# code has been modified from original
def get_predictions(df=test_dataset, loader=testing_loader):
    
    # put model in training mode
    model.eval()
    
    # GET WORD LABEL PREDICTIONS
    y_pred2 = []
    for batch in tqdm(loader, total=len(loader)):
        labels = inference(batch)
        y_pred2.extend(labels)

    final_preds2 = []
    for i in tqdm(range(len(df))):

        idx = df.id.values[i]
        #pred = [x.replace('B-','').replace('I-','') for x in y_pred2[i]]
        pred = y_pred2[i] # Leave "B" and "I"
        preds = []
        j = 0
        while j < len(pred):
            cls = pred[j]
            if cls == 'O': j += 1
            else: cls = cls.replace('B','I') # spans start with B
            end = j + 1
            while end < len(pred) and pred[end] == cls:
                end += 1
            
            if cls != 'O' and cls != '' and end - j > 7:
                final_preds2.append((idx, cls.replace('I-',''),
                                     ' '.join(map(str, list(range(j, end))))))
        
            j = end
        
    oof = pd.DataFrame(final_preds2)
    oof.columns = ['id','class','predictionstring']

    return oof

In [ ]:
# from Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [ ]:
if COMPUTE_VAL_SCORE: # note this doesn't run during submit
    # VALID TARGETS
    valid = train_df.loc[train_df['id'].isin(IDS[valid_idx])]

    # OOF PREDICTIONS
    oof = get_predictions(test_dataset, testing_loader)

    # COMPUTE F1 SCORE
    f1s = []
    CLASSES = oof['class'].unique()
    print()
    for c in tqdm(CLASSES):
        pred_df = oof.loc[oof['class']==c].copy()
        gt_df = valid.loc[valid['discourse_type']==c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        print(c, f1)
        f1s.append(f1)
    print()
    print('Overall', np.mean(f1s))
    print()

# Infer Test Data and Write Submission CSV
We will now infer the test data and write submission CSV

In [ ]:
sub = get_predictions(test_texts, test_texts_loader)
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)

# Test

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
import tokenizers
print(tokenizers.__version__)

In [ ]:
print(torch. __version__)

In [ ]:
a = np.array([[[1,2]]])

In [ ]:
a.shape

In [ ]:
a[0][0][0]

In [ ]:
a.nonzero()